In [1]:
import os
import pandas as pd
import scipy.stats as stats
import numpy as np
from itertools import combinations
from datetime import datetime

In [2]:
script_path = os.getcwd()
datapath = os.path.join(script_path,'data')
resultpath = os.path.join(script_path,'results')
most_likely = pd.read_csv(os.path.join(datapath,'most_likely.tsv'),delimiter='\t',header=0)
least_likely = pd.read_csv(os.path.join(datapath,'least_likely.tsv'),delimiter='\t',header=0)

In [3]:
## load parameters into dictionaries
resultsdict = {'most_likely':most_likely,'least_likely':least_likely}
grouptype = {'selection_only':['selection'],'scope':['scope','selection'],
             'domain':['domain','selection'],'round':['round','selection']}


In [4]:
def get_basic_stats(resultsdict,grouptype,resultopt,aggname,export='y'):
    aggoption = grouptype[aggname]
    results2test = resultsdict[resultopt]
    tmpselect = results2test.groupby(aggoption)['participants'].sum().reset_index(name='total votes')
    totals = selection_only['total votes'].sum()
    tmpselect['selection mean'] = tmpselect['total votes']/totals
    tmpselect['per test mean'] = results2test.groupby(aggoption)['participants'].mean().tolist()
    tmpselect['per test stdev'] = results2test.groupby(aggoption)['participants'].std().tolist()
    tmpselect['per test stderr'] = results2test.groupby(aggoption)['participants'].sem().tolist()
    tstamp = datetime.now()
    sttstamp = tstamp.strftime("%Y-%m-%d")
    if export == 'y':
        tmpselect.to_csv(os.path.join(resultpath,sttstamp+'_'+resultopt+'_'+aggname+'.tsv'),sep='\t',header=True)
    else:
        return tmpselect

In [5]:
def run_all_basic_stats(resultsdict,grouptype,export='y'):
    resultoptlist = list(resultsdict.keys())
    aggnamelist = list(grouptype.keys())
    for resultopt in resultoptlist:
        for aggname in aggnamelist:
            get_basic_stats(resultsdict,grouptype,resultopt,aggname,export)

In [ ]:
run_all_basic_stats(resultsdict,grouptype,export='y')

In [6]:
def run_t_tests(resultsdict,grouptype,resultopt,export='y'):
    tstamp = datetime.now()
    sttstamp = tstamp.strftime("%Y-%m-%d")
    aggopt = 'selection_only'
    aggname = grouptype[aggopt]
    results2test = resultsdict[resultopt]
    nonans = results2test['selection'].dropna()
    combilist = nonans.unique().tolist()
    selection_dict = {}
    for eachselect in combilist:
        selection_dict[eachselect] = np.array(results2test['participants'].loc[results2test['selection']==eachselect].tolist())
    tmpresult = []
    for combo in combinations(combilist, 2):  
        a1 = selection_dict[combo[0]]
        a2 = selection_dict[combo[1]]
        tmpstat, tmp_p = stats.ttest_ind(a1, a2)
        tmpdict = {"function 1":combo[0], "function 2":combo[1],
                   "mean 1": a1.mean(), "mean 2": a2.mean(),
                   "stdev 1": a1.std(), "stdev 2": a2.std(),
                   "t-test": tmpstat, "p-val": tmp_p,
                   "aggregation approach": aggopt, "run-date":sttstamp}
        tmpresult.append(tmpdict)

    statisticsdf = pd.DataFrame(tmpresult)
    print(statisticsdf.head(n=2))
    if export == 'y':
        statisticsdf.to_csv(os.path.join(resultpath,sttstamp+'_user_choice_only_'+resultopt+'.tsv'),sep='\t',header=True)
    else:
        return statisticsdf

In [7]:
run_t_tests(resultsdict,grouptype,'most_likely',export='y')

  function 1 function 2    mean 1    mean 2   stdev 1   stdev 2    t-test  \
0         c1         c3  1.962963  0.869565  1.170042  0.946589  3.518481   
1         c1         c2  1.962963  0.958333  1.170042  0.934486  3.293725   

      p-val aggregation approach    run-date  
0  0.000961       selection_only  2024-04-10  
1  0.001841       selection_only  2024-04-10  


C:\Users\gtsueng\AppData\Local\Temp\ipykernel_1684\1944825550.py:16: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  tmpstat, tmp_p = stats.ttest_ind(a1, a2)
C:\Users\gtsueng\Anaconda3\envs\nde\lib\site-packages\scipy\stats\_stats_py.py:1214: RuntimeWarning: divide by zero encountered in divide
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero
C:\Users\gtsueng\Anaconda3\envs\nde\lib\site-packages\scipy\stats\_stats_py.py:1214: RuntimeWarning: invalid value encountered in scalar multiply
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero


### Test components of the script

In [ ]:
tstamp = datetime.now()
print(tstamp.strftime("%Y-%m-%d"))

In [ ]:
## Get the Totals, Mean, Standard Deviation, Standard Error based on groupings by user selection
resultopt = 'most_likely'
aggopt = 'selection_only'
aggname = grouptype[aggopt]
results2test = resultsdict[resultopt]
tmpselect = results2test.groupby(aggname)['participants'].sum().reset_index(name='total votes')
totals = selection_only['total votes'].sum()
print(totals)
tmpselect['selection mean'] = tmpselect['total votes']/totals
tmpselect['per test mean'] = results2test.groupby(grouptype['selection_only'])['participants'].mean().tolist()
tmpselect['per test stdev'] = results2test.groupby(grouptype['selection_only'])['participants'].std().tolist()
tmpselect['per test stderr'] = results2test.groupby(grouptype['selection_only'])['participants'].sem().tolist()
print(tmpselect.head(n=2))

In [ ]:
## Perform single t-tests
resultopt = 'most_likely'
aggopt = 'selection_only'
aggname = grouptype[aggopt]
results2test = resultsdict[resultopt]

c1 = np.array(results2test['participants'].loc[results2test['selection']=='c1'].tolist())
c2 = np.array(results2test['participants'].loc[results2test['selection']=='c2'].tolist())
c3 = np.array(results2test['participants'].loc[results2test['selection']=='c3'].tolist())
c4 = np.array(results2test['participants'].loc[results2test['selection']=='c4'].tolist())

print("c1 vs c2: ",stats.ttest_ind(c1, c2))
print("c2 vs c3: ",stats.ttest_ind(c2, c3))
print("c1 vs c3: ",stats.ttest_ind(c1, c3))
print("c1 vs c4: ",stats.ttest_ind(c1, c4))
print("c2 vs c4: ",stats.ttest_ind(c2, c4))
print("c3 vs c4: ",stats.ttest_ind(c3, c4))

In [ ]:
statistic, pvalue = stats.ttest_ind(c3, c4)
print(len(c3), len(c4), statistic,pvalue)
print(c3.mean(),c3.std())

In [ ]:
#### automate comparisons and t-tests

## get the unique combis selected
nonans = results2test['selection'].dropna()
combilist = nonans.unique().tolist()
print(combilist)

## create a dictionary of the combis and the np arrays of the result dfs
selection_dict = {}
for eachselect in combilist:
    selection_dict[eachselect] = np.array(results2test['participants'].loc[results2test['selection']==eachselect].tolist())

## iterate through different combinations of the combis, run t-tests, and store the results
tmpresult = []
for combo in combinations(combilist, 2):  # 2 for pairs, 3 for triplets, etc
    a1 = selection_dict[combo[0]]
    a2 = selection_dict[combo[1]]
    tmpstat, tmp_p = stats.ttest_ind(a1, a2)
    tmpdict = {"function 1":combo[0], "function 2":combo[1],
               "mean 1": a1.mean(), "mean 2": a2.mean(),
               "stdev 1": a1.std(), "stdev 2": a2.std(),
               "t-test": tmpstat, "p-val": tmp_p,
               "aggregation approach": "selection only"}
    tmpresult.append(tmpdict)

statisticsdf = pd.DataFrame(tmpresult)
print(statisticsdf.head(n=2))
#statisticsdf.to_csv(os.path.join(resultpath,'user_choice_only_most.tsv'),sep='\t',header=True)
statisticsdf.to_csv(os.path.join(resultpath,'user_choice_only_least.tsv'),sep='\t',header=True)

### Investigate effect of different groupings (domain or scope) on selection

In [ ]:
## Subset and then perform t-tests
tmpgroup = results2test.loc[results2test['scope']=='broad']
tmpgroup2 = results2test.loc[results2test['scope']=='somewhat specific']
c1_broad = np.array(tmpgroup['participants'].loc[tmpgroup['selection']=='c1'].tolist())
c2_broad = np.array(tmpgroup['participants'].loc[tmpgroup['selection']=='c2'].tolist())
c1_ss = np.array(tmpgroup2['participants'].loc[tmpgroup2['selection']=='c1'].tolist())
c2_ss = np.array(tmpgroup2['participants'].loc[tmpgroup2['selection']=='c2'].tolist())
print("broad terms, c1 vs c2: ",stats.ttest_ind(c1_broad, c2_broad))
print("somewhat specific, c1 vs c2: ",stats.ttest_ind(c1_ss, c2_ss))

In [ ]:
## Perform paired t-test based on grouping
tmpgroup = results2test.groupby(['selection','scope'])['participants'].sum().reset_index(name='votes')
print(tmpgroup.head(n=2))
c1_group = np.array(tmpgroup['votes'].loc[tmpgroup['selection']=='c1'].tolist())
c2_group = np.array(tmpgroup['votes'].loc[tmpgroup['selection']=='c2'].tolist())
## create arrays that follow a specific order to ensure proper pairing

## Conduct paired t-test
tmpstat,tmp_p = stats.ttest_rel(c1_group,c2_group)
print(tmpstat,tmp_p)